# Chemprot: Bert NER on SageMaker using PyTorch

This uses the chemprot chemical protien names corpus in https://biocreative.bioinformatics.udel.edu/news/corpora/chemprot-corpus-biocreative-vi/






In [1]:
import sys, os
import logging

sys.path.append("src")

logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Bucket and role set up

In [2]:
import boto3

#from sagemaker import get_execution_role

import sagemaker.session 
sm_session = sagemaker.session.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')

# role=get_execution_role()
role ="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20181222T162635".format(account_id)


In [3]:
data_bucket = "aegovan-data"

data_bucket_prefix = "chemprotner"

s3_uri_data = "s3://{}/{}".format(data_bucket, data_bucket_prefix)
s3_uri_train = "{}/{}".format(s3_uri_data, "train/chemprot_training_abstracts.tsv")
s3_uri_train_classes = "{}/{}".format(s3_uri_data, "train/chemprot_training_entities.tsv")

s3_uri_val = "{}/{}".format(s3_uri_data, "val/chemprot_development_abstracts.tsv")
s3_uri_val_classes = "{}/{}".format(s3_uri_data, "val/chemprot_development_entities.tsv")



s3_uri_test = "s3://{}/{}/test data".format(data_bucket, data_bucket_prefix , "test.in")

s3_output_path = "s3://{}/{}/output".format(data_bucket, data_bucket_prefix)
s3_code_path = "s3://{}/{}/code".format(data_bucket, data_bucket_prefix)
s3_checkpoint = "s3://{}/{}/checkpoint".format(data_bucket, data_bucket_prefix)

## Train

This shows you how to train BERT on SageMaker using SPOT instances

In [4]:
inputs_full =  {
    "train" : s3_uri_train,
    "class" : s3_uri_train_classes,
    "val" : s3_uri_val,
    "valclass" : s3_uri_val_classes
}

inputs = inputs_full

In [5]:
sm_localcheckpoint_dir="/opt/ml/checkpoints/"

In [6]:
instance_type = "ml.p3.2xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [16]:
hp = {
"epochs" : 30,
"earlystoppingpatience" : 5,
# Increasing batch size might end up with CUDA OOM error, increase grad accumulation instead
"batch" : 8 * instance_type_gpu_map[instance_type],
"trainfile" :s3_uri_train.split("/")[-1],
"classfile":s3_uri_train_classes.split("/")[-1],
"valfile" :s3_uri_val.split("/")[-1],
"valclassfile":s3_uri_val_classes.split("/")[-1],
"datasetfactory":"datasets.chemprot_dataset_factory.ChemprotDatasetFactory",
# The number of steps to accumulate gradients for
"gradaccumulation" : 4,
"log-level":"INFO",
# This param depends on your model max pos embedding size or when large you might end up with CUDA OOM error    
"maxseqlen" : 512,
# Make sure the lr is quite small, as this is a pretrained model..
"lr":0.00001,
# Use finetuning (set to 1), if you only want to change the weights in the final classification layer.. 
"finetune": 0,
"checkpointdir" : sm_localcheckpoint_dir,
# Checkpoints once every n epochs
"checkpointfreq": 2,
"log-level" : "INFO"
}



In [17]:
hp

{'epochs': 30,
 'earlystoppingpatience': 5,
 'batch': 8,
 'trainfile': 'chemprot_training_abstracts.tsv',
 'classfile': 'chemprot_training_entities.tsv',
 'valfile': 'chemprot_development_abstracts.tsv',
 'valclassfile': 'chemprot_development_entities.tsv',
 'datasetfactory': 'datasets.chemprot_dataset_factory.ChemprotDatasetFactory',
 'gradaccumulation': 4,
 'log-level': 'INFO',
 'maxseqlen': 512,
 'lr': 1e-05,
 'finetune': 0,
 'checkpointdir': '/opt/ml/checkpoints/',
 'checkpointfreq': 2}

In [18]:
inputs

{'train': 's3://aegovan-data/chemprotner/train/chemprot_training_abstracts.tsv',
 'class': 's3://aegovan-data/chemprotner/train/chemprot_training_entities.tsv',
 'val': 's3://aegovan-data/chemprotner/val/chemprot_development_abstracts.tsv',
 'valclass': 's3://aegovan-data/chemprotner/val/chemprot_development_entities.tsv'}

In [19]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ]

In [20]:
# set True if you need spot instance
use_spot = True
train_max_run_secs =   2*24 * 60 * 60
spot_wait_sec =  5 * 60
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    # Use smaller dataset to run locally
    inputs = inputs_sample


In [12]:
job_type = "chemprot-ner-bert"
base_name = "{}".format(job_type)

In [21]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='main.py',
                    source_dir = 'src',
                    role=role,
                    framework_version ="1.4.0",
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters = hp,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    volume_size=30,
                    code_location=s3_code_path,
                    debugger_hook_config=False,
                    base_job_name =base_name,  
                    use_spot_instances = use_spot,
                    max_run =  train_max_run_secs,
                    max_wait = max_wait_time_secs,   
                    checkpoint_s3_uri=s3_checkpoint,
                    checkpoint_local_path=sm_localcheckpoint_dir
                    )

estimator.fit(inputs, wait=True)

2022-07-16 13:37:04,955 - sagemaker.image_uris - INFO - Defaulting to the only supported framework/algorithm version: latest.
2022-07-16 13:37:04,981 - sagemaker.image_uris - INFO - Ignoring unnecessary instance type: None.
2022-07-16 13:37:05,000 - sagemaker - INFO - Creating training-job with name: chemprot-ner-bert-2022-07-16-18-37-04-135
2022-07-16 18:37:05 Starting - Starting the training job...
2022-07-16 18:37:30 Starting - Preparing the instances for trainingProfilerReport-1657996624: InProgress
............
2022-07-16 18:39:40 Downloading - Downloading input data
2022-07-16 18:39:40 Training - Downloading the training image............
2022-07-16 18:41:41 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-07-16 18:41:44,135 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2022-07-16 18:41:44,160 sagemaker_pyt

2022-07-16 18:42:09,780 - transformers.file_utils - INFO - storing https://cdn.huggingface.co/bert-base-cased-pytorch_model.bin in cache at /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2022-07-16 18:42:09,780 - transformers.file_utils - INFO - creating metadata file for /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2022-07-16 18:42:09,780 - transformers.modeling_utils - INFO - loading weights file https://cdn.huggingface.co/bert-base-cased-pytorch_model.bin from cache at /root/.cache/torch/transformers/d8f11f061e407be64c4d5d7867ee61d1465263e24085cfa26abf183fdc830569.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2022-07-16 18:42:13,017 - transformers.modeling_utils - WARNING - Some weights of the model checkpoint at bert-base-cased were no


2022-07-16 19:07:57 Uploading - Uploading generated training model
2022-07-16 19:07:57 Completed - Training job completed
ProfilerReport-1657996624: IssuesFound
Training seconds: 1700
Billable seconds: 510
Managed Spot Training savings: 70.0%


## Deploy BERT model

#### Inference container
Ideally the server containing should already have all the required dependencies installed to reduce start up time and ensure that the runtime enviornment is consistent. This can be implemented using a custom docker image.

But for this demo, to simplify, we will let the Pytorch container script model install the dependencies during start up. As a result, you will see some of the initial ping requests fail, until all dependencies are installed.


In [ ]:
import sagemaker
training_job = "bc2-ner-bert-2020-10-04-14-09-30-400"
estimator = sagemaker.estimator.Estimator.attach(training_job)

In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
role = role

model_uri = estimator.model_data

model = PyTorchModel(model_data=model_uri,
                     role=role,
                     framework_version='1.4.0',
                     py_version = "py3",
                     entry_point='serve.py',
                     source_dir='src'
                    
                    )

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

### Invoke API

In [ ]:
class Predictor:
    
    def serialize(self, x):
        return x
    
    def deserialize(self,x, content_type):
        payload_bytes = json.loads( x.read().decode("utf-8") )
        return payload_bytes

In [ ]:
#predictor= sagemaker.predictor.Predictor("end")
predictor.serializer = Predictor()
predictor.deserializer = Predictor()


In [ ]:
import json

from datasets.biocreative_dataset import BiocreativeDataset
from datasets.biocreative_ner_label_mapper import BiocreativeNerLabelMapper

def chunk(l, size=5):
    for i in range(0, len(l),size):
        yield l[i:i+size]

with open("tmp/test.in", "r") as f:
    d = []
    ids =[]
    for l in f.readlines():
        l = l.rstrip("\n")
        (id, text) = l.split(" ")[0], " ".join(l.split(" ")[1:])
        d.append(text)
        ids.append(id)
        
    
    
label_mapper = BiocreativeNerLabelMapper()

id_chucks = list(chunk(ids))
result = []
for (i, data) in enumerate(chunk(d)):

    data_bytes=("\n".join(data)).encode("utf-8")
    response  = predictor.predict(data_bytes,  
                                    initial_args={ "Accept":"text/json", "ContentType" : "text/csv" }
                                       )

    assert len(response) == len(data), "Data size {} doesnt match result size {}".format(len(r), len(d))



    for ri, r in enumerate(response):
        doc_id = id_chucks[i][ri]

        result.append({"docid":doc_id, "text": data[ri], "entities_detected": r })
        
    
    
with open("result.json", "w") as f:
    json.dump( result, f)




## Delete endpoint

In [ ]:
predictor.delete_endpoint()